In [40]:
import pandas as pd 
import numpy as np 
import tensorflow as tf
import matplotlib.pyplot as plt 

from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, TimeDistributed, Conv1D, MaxPooling1D, Flatten, Bidirectional, Input, Flatten, Activation, Reshape, RepeatVector, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import stat

In [41]:
df = pd.read_csv('./target_lgb_filled.csv').drop('Unnamed: 0' , axis = 1)

In [42]:
df

,0,1,2,3,4,5,6,7,8,9,...,3454,3455,3456,3457,3458,3459,3460,3461,3462,3463
0,0.121164,-0.300875,-0.231040,0.139263,-0.016268,0.568807,-1.064780,-0.531940,1.505904,-0.260731,...,0.016876,0.211660,0.302557,0.003156,0.114607,-0.392297,-0.877746,-0.284696,0.202003,0.009833
1,0.094861,-0.917045,-0.472108,0.110934,0.099531,-0.147971,-0.372692,-0.105693,0.622500,-0.400038,...,0.068460,0.315893,-0.560079,0.250396,1.318857,-0.227782,-0.684049,-0.894825,-0.286612,0.115699
2,0.143137,-0.480234,-0.661659,0.159401,0.035933,0.243674,0.318899,-0.260137,-0.610705,-1.030857,...,0.016876,0.230765,-0.305467,2.031675,-0.040981,-0.018971,-0.250995,-0.323800,0.300915,0.071693
3,0.170280,-0.323562,-0.055215,0.228169,0.075363,1.816745,-0.711446,-0.640987,5.271096,-0.636719,...,-0.000747,0.258393,-0.730791,0.857357,0.386379,-0.708491,-0.165561,0.836601,0.076417,0.046693
4,0.149331,2.494479,0.341267,0.063783,0.121211,0.470476,-1.581650,-0.592970,0.288508,-0.435524,...,0.068460,0.247359,-0.656495,1.897659,-1.476258,-0.210125,-0.206145,0.126859,-0.387297,-0.181363
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1206,0.579622,0.078338,-0.361899,1.624498,0.968079,-0.353440,-0.005973,-0.665880,-0.344785,-0.818674,...,-0.074509,0.608381,-0.872156,-0.768846,-0.421168,0.045190,0.227717,-0.811419,0.292205,-0.049360
1207,0.064896,-0.290113,-0.688692,-0.070665,-0.514005,-0.095650,0.037676,0.095912,-0.161798,-0.489038,...,0.163917,0.666394,-0.918939,1.095260,-0.200416,-0.104977,-0.230372,-1.410046,0.651031,0.526346
1208,2.204664,-0.097136,0.007041,1.532567,-0.207239,0.287350,0.139894,0.567659,-1.444997,0.313066,...,0.351041,0.511106,0.261462,-0.467051,0.019905,0.577800,0.345404,-0.477728,-0.422950,1.492833
1209,1.037581,-0.080295,-0.082326,0.384600,0.318535,-0.187984,-0.007878,-0.586860,-0.371770,0.534930,...,-0.029728,0.990797,-0.154512,-0.254095,-0.200416,-0.164939,0.644711,-0.119869,0.513739,-0.386294


In [43]:
investment_ticker = df.iloc[: , 0 ]

In [44]:
from keras.preprocessing.sequence import TimeseriesGenerator

df_gen =  TimeseriesGenerator(np.array(df), np.array(df) , length= 5 , sampling_rate=1  ,batch_size = len(df)  )

In [45]:
investment_ticker = df.iloc[: , 0 ]
FORECAST_RANGE = 1  
LOOK_BACK = 5 
n_features= 1  

epochs = 50
batch_size = 32
validation = 0.1

model_enc_dec = Sequential()
model_enc_dec.add(LSTM(100, activation='relu', input_shape=(LOOK_BACK, n_features)))
model_enc_dec.add(RepeatVector(FORECAST_RANGE))
model_enc_dec.add(LSTM(100, activation='relu', return_sequences=True))
model_enc_dec.add(TimeDistributed(Dense(n_features)))
model_enc_dec.compile(optimizer='adam', loss='mse')
plot_model(model=model_enc_dec, show_shapes=True)





You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


In [59]:
x, y = df_gen[0]
print(x.shape , y.shape)

print(x[:,:,0])
    

(1206, 5, 3464) (1206, 3464)
[[ 0.12116425  0.09486069  0.14313655  0.17028032  0.14933099]
 [ 0.09486069  0.14313655  0.17028032  0.14933099  0.02324629]
 [ 0.14313655  0.17028032  0.14933099  0.02324629  0.10021492]
 ...
 [-2.6210136   0.12961906  1.1389745   0.57962227  0.06489602]
 [ 0.12961906  1.1389745   0.57962227  0.06489602  2.2046645 ]
 [ 1.1389745   0.57962227  0.06489602  2.2046645   1.0375807 ]]


In [61]:
pearson = [] 
x_0 , y_0 = df_gen[0] 

for i in range(x_0.shape[2]) : 
    x  = x_0[:,:,i]
    y = y_0[:,i]
    X_train , X_test , y_train , y_test = x[:1000] , x[1000:], y[:1000] , y[1000:]
    history = model_enc_dec.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=validation,callbacks=[early_stopping_callback, rlrop_callback])
    yhat = model_enc_dec.predict(X_test, verbose=0)
    pred = list(yhat)
    hehe = np.squeeze(np.asarray(y_test))
    haha = np.squeeze(np.asarray(pred))
    result = np.all(haha == haha[0])
    if result:
        print('N')       
    pearson_score = stats.pearsonr(haha, hehe)[0]
    pearson.append(pearson_score)


Epoch 1/50
29/29 [==============================] - 1s 21ms/step - loss: 0.1701 - val_loss: 1.0909 - lr: 0.0010
Epoch 2/50
29/29 [==============================] - 1s 22ms/step - loss: 0.1699 - val_loss: 1.0936 - lr: 0.0010
Epoch 3/50
29/29 [==============================] - 1s 20ms/step - loss: 0.1701 - val_loss: 1.0951 - lr: 0.0010
Epoch 4/50
29/29 [==============================] - 1s 21ms/step - loss: 0.1700 - val_loss: 1.0900 - lr: 0.0010
Epoch 5/50
29/29 [==============================] - 1s 21ms/step - loss: 0.1698 - val_loss: 1.0930 - lr: 0.0010
Epoch 6/50
29/29 [==============================] - 1s 21ms/step - loss: 0.1700 - val_loss: 1.0912 - lr: 0.0010
Epoch 7/50
29/29 [==============================] - 1s 21ms/step - loss: 0.1699 - val_loss: 1.0938 - lr: 0.0010
Epoch 8/50
29/29 [==============================] - 1s 21ms/step - loss: 0.1699 - val_loss: 1.0923 - lr: 0.0010
Epoch 9/50
29/29 [==============================] - 1s 22ms/step - loss: 0.1703 - val_loss: 1.0902 - lr:

KeyboardInterrupt: 

In [66]:
sum =  0 
for i in pearson : 
    sum+=abs(i)
print(sum / len(pearson))

0.07877054571739363


In [20]:
X_train , X_test , y_train , y_test = x[:1000] , x[1000:], y[:1000] , y[1000: ]

In [21]:

early_stopping_callback = EarlyStopping(
 monitor='val_loss',
 min_delta=0.005,
 patience=10,
 mode='min'
)
rlrop_callback = ReduceLROnPlateau(monitor='val_loss', factor=0.2, mode='min', patience=3, min_lr=0.001)

In [29]:
from scipy import stats

yhat.reshape(1,-1)
pred = list(yhat)
hehe = np.squeeze(np.asarray(y_test))
haha = np.squeeze(np.asarray(pred))
result = np.all(haha == haha[0])
if result:
      print('N')       
pearson_score = stats.pearsonr(haha, hehe)[0]
print(pearson_score)

-0.07989429274734065
